# 1. Análisis de componentes principales



**Objetivo:** Transofrmar el espacio de características a un espacio con componentes principales.


**Información de las características**
Este dataset contiene imágenes que pertenecen all dataset de EuroSat. Hay 10 folders:
* 0 AnnualCrop
* 1 Forest
* 2 HerbaceousVegatation
* 3 Highway
* 4 Industrial
* 5 Pasture
* 6 PermanentCrop
* 7 Residential
* 8 River
* 9 SeaLake


**Número de instancias:** 27000

# 2. Autenticación a Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U kaleido
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

# 3. Importando módulos

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.io as pio
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

# 4. Lectura de archivo

In [ ]:
train_path = r'/content/drive/Shareddrives/Data Science para Geociencias/4. Selección de caracteristicas/Resultados/EUROSAT_TRAIN_FEAT.csv'
results_path = r'/content/drive/Shareddrives/Data Science para Geociencias/4. Selección de caracteristicas/Resultados'

In [ ]:
train_df = pd.read_csv(train_path)
train_df.head()

,histogram_0,histogram_1,histogram_2,histogram_3,histogram_4,histogram_5,histogram_6,histogram_7,histogram_8,histogram_9,histogram_10,histogram_11,histogram_12,histogram_13,histogram_14,histogram_15,histogram_16,histogram_17,histogram_18,histogram_19,histogram_20,histogram_21,histogram_22,histogram_23,histogram_24,histogram_25,histogram_26,histogram_27,histogram_28,histogram_29,histogram_30,histogram_31,histogram_32,histogram_33,histogram_34,histogram_35,histogram_36,histogram_37,histogram_38,histogram_39,...,histogram_493,histogram_494,histogram_495,histogram_496,histogram_497,histogram_498,histogram_499,histogram_500,histogram_501,histogram_502,histogram_503,histogram_504,histogram_505,histogram_506,histogram_507,histogram_508,histogram_509,histogram_510,histogram_511,hal_0,hal_1,hal_2,hal_3,hal_4,hal_5,hal_6,hal_7,hal_8,hal_9,hal_10,hal_11,hal_12,hum_0,hum_1,hum_2,hum_3,hum_4,hum_5,hum_6,label
0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012692,8.819525,0.606407,11.202328,0.411772,112.869246,35.989785,4.587024,6.990196,0.002292,2.686005,-0.132405,0.783513,0.002973,6.369633e-10,1.516469e-13,7.302349e-13,9.142081e-26,8.576981e-18,-2.251498e-25,Forest
1,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016780,6.737624,0.523612,7.070943,0.395274,113.673990,21.546147,4.192914,6.515763,0.002552,2.556843,-0.087996,0.659967,0.002942,2.693041e-10,9.678997e-15,7.353245e-13,1.652979e-26,1.191183e-17,5.979175e-26,Forest
2,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025098,4.159946,0.587513,5.044265,0.489949,105.225032,16.017115,4.011338,5.961651,0.003509,2.247017,-0.124186,0.727920,0.003155,4.341931e-10,1.849652e-12,2.115307e-13,-1.071065e-25,3.151259e-19,7.768625e-26,Forest
3,0.0,0.0,0.0,0.000718,0.00395,0.001436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022486,50.834805,0.744700,99.422595,0.436835,117.787398,346.855576,4.579942,6.767477,0.001086,2.938769,-0.190203,0.866096,0.002811,4.023647e-09,7.724173e-12,3.490115e-12,-1.568084e-23,6.745461e-17,9.082255e-24,Forest
4,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007640,12.731279,0.589228,15.496675,0.334405,113.976952,49.255423,4.815528,7.530149,0.001799,2.969903,-0.106116,0.746747,0.002936,4.838209e-10,3.726097e-12,5.626374e-12,2.496585e-23,1.228961e-16,-6.352761e-24,Forest


# 5. Escalamiento

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_df.loc[:, train_df.columns != 'label'] = scaler.fit_transform(train_df.loc[:, train_df.columns != 'label'])

In [ ]:
train_df.head()

,histogram_0,histogram_1,histogram_2,histogram_3,histogram_4,histogram_5,histogram_6,histogram_7,histogram_8,histogram_9,histogram_10,histogram_11,histogram_12,histogram_13,histogram_14,histogram_15,histogram_16,histogram_17,histogram_18,histogram_19,histogram_20,histogram_21,histogram_22,histogram_23,histogram_24,histogram_25,histogram_26,histogram_27,histogram_28,histogram_29,histogram_30,histogram_31,histogram_32,histogram_33,histogram_34,histogram_35,histogram_36,histogram_37,histogram_38,histogram_39,...,histogram_493,histogram_494,histogram_495,histogram_496,histogram_497,histogram_498,histogram_499,histogram_500,histogram_501,histogram_502,histogram_503,histogram_504,histogram_505,histogram_506,histogram_507,histogram_508,histogram_509,histogram_510,histogram_511,hal_0,hal_1,hal_2,hal_3,hal_4,hal_5,hal_6,hal_7,hal_8,hal_9,hal_10,hal_11,hal_12,hum_0,hum_1,hum_2,hum_3,hum_4,hum_5,hum_6,label
0,0.0,0.0,0.0,0.00000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024280,0.005422,0.600306,0.001954,0.447481,0.115357,0.001613,0.462514,0.491858,0.177397,0.326815,0.783520,0.764189,0.576694,0.001290,0.000088,0.000462,0.902864,0.063770,0.818836,Forest
1,0.0,0.0,0.0,0.00000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032223,0.004092,0.514067,0.001221,0.427120,0.117643,0.000959,0.407260,0.447951,0.197881,0.302561,0.858441,0.629037,0.568891,0.000545,0.000006,0.000465,0.902864,0.063773,0.818836,Forest
2,0.0,0.0,0.0,0.00000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048387,0.002445,0.580627,0.000862,0.543958,0.093642,0.000708,0.381803,0.396671,0.273576,0.244384,0.797386,0.703374,0.623777,0.000879,0.001069,0.000134,0.902864,0.063763,0.818836,Forest
3,0.0,0.0,0.0,0.00075,0.0041,0.001614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043312,0.032265,0.744353,0.017589,0.478410,0.129328,0.015698,0.461521,0.471247,0.082033,0.374278,0.686013,0.854530,0.534996,0.008148,0.004465,0.002208,0.902856,0.063820,0.818851,Forest
4,0.0,0.0,0.0,0.00000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014464,0.007921,0.582412,0.002715,0.352002,0.118504,0.002214,0.494551,0.541829,0.138432,0.380124,0.827871,0.723970,0.567159,0.000980,0.002154,0.003559,0.902877,0.063867,0.818826,Forest


# 6. PCA

#### a) PCA con 2 componentes

In [ ]:
pca = PCA(n_components=2)
pc = pca.fit_transform(train_df.iloc[:,:-1])
pca_df = pd.DataFrame(data = pc
             , columns = ['PC 1', 'PC 2'])
pca_df = pd.concat([pca_df, train_df[['label']]], axis = 1)

Varianza acumulada alcanzada

In [ ]:
print(np.sum(pca.explained_variance_ratio_))

0.3931316963910484


Visualización del 2D

In [ ]:
fig = px.scatter(data_frame=pca_df,
                    x='PC 1',
                    y='PC 2',
                    color='label',
                    color_continuous_scale=px.colors.sequential.Rainbow,
                    template='seaborn',
                    title='PCA 2D',
                    width=1000,
                    height=500)
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=25))
fig.update_traces(marker=dict(size=4),
                  selector=dict(mode='markers'))
pio.show(fig)
fig.write_html("pca_2.html")
fig.write_image("pca_2.png")

#### b) PCA con 3 componentes *principales* **texto en negrita**

In [ ]:
pca = PCA(n_components=3)
pc = pca.fit_transform(train_df.iloc[:,:-1])
pca_df = pd.DataFrame(data = pc
             , columns = ['PC 1', 'PC 2', 'PC 3'])
pca_df = pd.concat([pca_df, train_df[['label']]], axis = 1)

Poder explicativo

In [ ]:
print(np.sum(pca.explained_variance_ratio_))

0.4724098589887687


Visualización de PCA con 3 componentes

In [ ]:
fig = px.scatter_3d(data_frame=pca_df,
                    x='PC 1',
                    y='PC 2',
                    z='PC 3',
                    color='label',
                    color_continuous_scale=px.colors.sequential.Plasma,
                    template='seaborn',
                    title='PCA 3D EuroSat',
                    width=1300,
                    height=500)
# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=25))
fig.update_traces(marker=dict(size=1),
                  selector=dict(mode='markers'))
pio.show(fig)
fig.write_html("pca_3.html")
fig.write_image("pca_3.png")

#### c) PCA con poder explicativo del 95%

In [ ]:
pca = PCA(0.95)
pc = pca.fit_transform(train_df.iloc[:,:-1])
df_pca_train = pd.DataFrame(data = pc,
                           columns=range(pc.shape[1]))
df_pca_train = pd.concat([df_pca_train, train_df[['label']]], axis = 1)


Poder explicativo:

In [ ]:
print('Número de componentes principales: %s'%len(pca.explained_variance_ratio_))
print('Varianza acumulada con %s componentes: %s'%(len(pca.explained_variance_ratio_),np.sum(pca.explained_variance_ratio_)))

Número de componentes principales: 40
Varianza acumulada con 40 componentes: 0.9519095155727268


In [ ]:
df_pca_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,label
0,0.794936,-0.309045,-0.034182,0.472750,0.104407,-0.103815,-0.128473,-0.061032,0.064459,0.072393,0.088103,0.056799,0.042715,-0.005131,0.003639,0.034863,-0.021275,-0.018063,-0.012143,-0.017243,-0.024100,0.022219,0.013643,-0.010604,0.000754,-0.002111,-0.005386,0.001826,-0.000857,-0.001745,0.002240,-0.011140,0.032160,-0.007401,0.001419,0.025569,-0.000670,0.015193,0.004009,-0.006657,Forest
1,0.902883,-0.171158,0.082851,0.504755,-0.139262,-0.128761,-0.117520,-0.183206,0.052243,0.073681,0.072478,0.086872,0.062678,0.026526,-0.028415,-0.004697,0.015997,-0.031355,-0.011097,0.008971,0.008099,0.004882,0.004061,-0.008497,-0.007257,-0.002113,0.007803,-0.006287,0.000254,-0.000380,-0.005413,-0.008851,0.061036,-0.020636,-0.003227,-0.002216,0.011670,0.017865,-0.002477,0.010373,Forest
2,0.988506,-0.217424,0.033294,0.475327,-0.042156,-0.084267,-0.081708,-0.049299,0.047954,0.031502,0.058258,0.060630,0.038480,0.003708,-0.007224,0.020500,0.031670,-0.001525,-0.012512,-0.006399,-0.007664,-0.004505,0.008967,-0.002918,0.007103,0.000707,-0.009001,0.014962,-0.006745,-0.008176,-0.007397,0.001147,0.014129,-0.002226,-0.003956,0.013514,-0.006779,0.013776,-0.002941,-0.010047,Forest
3,0.713198,-0.308842,-0.103465,0.525621,-0.021424,-0.066784,-0.009050,0.004468,0.023701,0.029169,0.023041,0.025966,0.014983,-0.014204,0.004963,0.020652,-0.031775,0.000753,-0.003119,-0.015785,-0.023915,0.025521,0.007246,-0.010937,0.003210,-0.005477,-0.004627,-0.007053,-0.006959,0.001921,0.013111,-0.015139,0.033895,-0.008116,0.016088,0.051263,0.010705,0.032487,0.020406,-0.017095,Forest
4,0.682355,-0.368111,-0.044976,0.383385,0.298409,-0.134437,-0.238336,-0.093024,0.096360,0.121753,0.128168,0.066764,0.062802,-0.005895,-0.001787,0.051066,-0.041069,-0.032094,-0.014380,-0.024295,-0.032053,0.035964,0.020359,-0.015014,0.017690,0.001503,-0.003326,0.001978,0.009980,-0.007524,-0.001443,-0.011687,0.052227,-0.013729,-0.003570,0.010927,0.002747,0.022990,0.000192,-0.002363,Forest


Guradando el dataframe generado

In [ ]:
df_pca_train.to_csv(os.path.join(results_path,'EUROSAT_TRAIN_PCA.csv'))